In [1]:
import os
import glob
import ast
import pickle
import pandas as pd
import numpy as np
from Bio import SeqIO

In [2]:
file = os.path.join("/home/ubuntu/data/dmel_seq", "dmel_only_with_length.fa")
output_folder_path = "/home/ubuntu/data/dmel_seq"
# NOTE: the buffer file need not be created beforehands. Just write the path
#       and the file name here. The file would be created by the system.

In [3]:
base_pairs = {'A': 1, 
              'C': 2,
              'G': 3,
              'T': 4,
              'a': 1,
              'c': 2,
              'g': 3,
              't': 4,
              'n': 0,
              'N': 0}

In [4]:
data = list(SeqIO.parse(file,"fasta"))

In [5]:
# min and max len
min_len = min([int(seq.description.split('|')[-1]) for seq in data])
max_len = max([int(seq.description.split('|')[-1]) for seq in data])
min_len, max_len

(116, 3388)

In [8]:
mu_len = np.mean([int(seq.description.split('|')[-1]) for seq in data])
median_len = np.median([int(seq.description.split('|')[-1]) for seq in data])
mu_len, median_len

(2030.5040557667935, 2135.0)

In [6]:
data_x, data_y = [], []
for seq in data:
    regionid, expressed, speciesid, strand, _ = seq.description.split('|')
    y = int(expressed)
    x_len = len(seq.seq)
    x = [base_pairs[char] for char in seq.seq]
    x_comp = [base_pairs[char] for char in seq.seq.complement()]
    while x_len < median:
        x.append(0)
        x_comp.append(0)
        x_len += 1
    data_x.append(x)
    data_x.append(x_comp)
    data_y.append(y)
    data_y.append(y)

tokenized_x, tokenized_y = np.array(data_x), np.array(data_y)

In [11]:
tokenized_x.shape, tokenized_y.shape

((15780, 3388), (15780,))

In [9]:
with open(os.path.join(output_folder_path, 'tokenized_x.np'), mode='wb') as output:
    pickle.dump(tokenized_x, output)
with open(os.path.join(output_folder_path, 'tokenized_y.np'), mode='wb') as output:
    pickle.dump(tokenized_y, output)